In [1]:
# make sure working directory is root of repository
%cd ..
%pwd

/Users/lydiareader/git/dcds-500-project-2


'/Users/lydiareader/git/dcds-500-project-2'

In [2]:
from shopping_environment import ShoppingEnvironment
import pandas as pd

## Setup Environment

In [3]:
# parameters
num_shoes        = 1000

# number of each consumer type
num_average     = 9000
num_abnormal    = 1000
num_special     = 250
num_wealthy     = 1000
num_influencers = 500
num_resellers   = 2500
num_total       = num_average + num_abnormal + num_special + num_wealthy + num_influencers + num_resellers

# initialize environment
env = ShoppingEnvironment(
    mean_desire = 0.5,
    std_desire = 0.2,
    desire_threshold = 0.5,
    average_mean_money = 250,
    average_std_money = 15,
    rich_mean_money = 1000,
    rich_std_money = 250,
    average_prob_loyal = 0.5,
    rich_prob_loyal = 0.8,
    reseller_prob_loyal = 0.9,
    num_shoes = num_shoes,
    price = 250,
    num_average = num_average,
    num_abnormal = num_abnormal,
    num_special = num_special,
    num_wealthy = num_wealthy,
    num_influencers = num_influencers,
    num_resellers = num_resellers
)

In [4]:
def get_utilities(df):
    # buyer and seller utility for each simulation
    got_shoes = df[df['shoes_acquired'] >= 1]
    print("Consumer utility: ", got_shoes['desire'].mean())     # buyer utility is average desire of those who purchased shoes
    print("Seller utility: ", got_shoes['influence'].mean())    # seller utility is average influence of those who purchased shoes 


def group_fairness(df):
    ret_dict = {}
    got_shoes = df[df['shoes_acquired'] >= 1]
    ret_dict['Consumer utility'] = got_shoes['desire'].mean()
    ret_dict['Seller utility'] = got_shoes['influence'].mean()
    amount_con_types = {
        'average': num_average,
        'abnormal': num_abnormal,
        'special': num_special,
        'wealthy': num_wealthy,
        'influencer': num_influencers,
        'reseller': num_resellers
    }
    for con, amount in amount_con_types.items():
        ret_dict[f'{con}_num_got_shoes'] = len(got_shoes[got_shoes['identity'] == con])
        ret_dict[f'{con}_percent_got_shoes'] = len(got_shoes[got_shoes['identity'] == con]) / amount
        ret_dict[f'{con}_shoes_bought'] = df.loc[df['identity'] == con, 'shoes_acquired'].sum()
        ret_dict[f'{con}_shoes_per_person'] = df.loc[df['identity'] == con, 'shoes_acquired'].sum() / amount

    return ret_dict
        

## No Gaming

### Lottery
Every consumer has an equal chance of being selected to purchase shoes

In [5]:
env.restock(1000)
env.run_lottery_no_gaming()  # run lottery and get dataframe of all results
df = env.get_consumer_df()
lottery_no_gaming = pd.Series(group_fairness(df), name='lottery_no_gaming')
lottery_no_gaming

Consumer utility                  0.409377
Seller utility                    0.167213
average_num_got_shoes           377.000000
average_percent_got_shoes         0.041889
average_shoes_bought            552.000000
average_shoes_per_person          0.061333
abnormal_num_got_shoes           40.000000
abnormal_percent_got_shoes        0.040000
abnormal_shoes_bought            62.000000
abnormal_shoes_per_person         0.062000
special_num_got_shoes            12.000000
special_percent_got_shoes         0.048000
special_shoes_bought             24.000000
special_shoes_per_person          0.096000
wealthy_num_got_shoes            42.000000
wealthy_percent_got_shoes         0.042000
wealthy_shoes_bought             84.000000
wealthy_shoes_per_person          0.084000
influencer_num_got_shoes         23.000000
influencer_percent_got_shoes      0.046000
influencer_shoes_bought          46.000000
influencer_shoes_per_person       0.092000
reseller_num_got_shoes          116.000000
reseller_pe

### First Come First Served
With no gaming, those with highest desire "come first", and so those with the most desire get the sheos

In [6]:
env.restock(num_shoes = 1000)        # reset environment to 100 shoes with no one purchased yet
env.run_first_come_no_gaming()
df = env.get_consumer_df()
first_no_gaming = pd.Series(group_fairness(df), name='first_no_gaming')

### Invitation Only
All influencers are invited, and the rest of the spots are randomly selected from those with loyalty status

In [7]:
env.restock(num_shoes = 1000)
env.run_invitation_no_gaming()
df = env.get_consumer_df()
invite_no_gaming = pd.Series(group_fairness(df), name='invite_no_gaming')

In [8]:
no_gaming = pd.DataFrame([lottery_no_gaming, first_no_gaming, invite_no_gaming])

## With Gaming

### Lottery
Every consumer has an equal chance of being selected to purchase shoes

In [7]:
env.restock(1000)
env.run_lottery_with_gaming()  # run lottery and get dataframe of all results
df = env.get_fake_df()
lottery_with_gaming = pd.Series(group_fairness(df), name='lottery_with_gaming')
lottery_with_gaming

Consumer utility                  0.155560
Seller utility                    0.372709
average_num_got_shoes           115.000000
average_percent_got_shoes         0.012778
average_shoes_bought            197.000000
average_shoes_per_person          0.021889
abnormal_num_got_shoes           10.000000
abnormal_percent_got_shoes        0.010000
abnormal_shoes_bought            17.000000
abnormal_shoes_per_person         0.017000
special_num_got_shoes             3.000000
special_percent_got_shoes         0.012000
special_shoes_bought              6.000000
special_shoes_per_person          0.024000
wealthy_num_got_shoes            21.000000
wealthy_percent_got_shoes         0.021000
wealthy_shoes_bought             42.000000
wealthy_shoes_per_person          0.042000
influencer_num_got_shoes          3.000000
influencer_percent_got_shoes      0.006000
influencer_shoes_bought           6.000000
influencer_shoes_per_person       0.012000
reseller_num_got_shoes          339.000000
reseller_pe

### First Come First Served
With gaming, those with highest desire "come first", and so those with the most desire get the sheos

In [10]:
env.restock(num_shoes = 1000)        # reset environment to 100 shoes with no one purchased yet
env.run_first_come_with_gaming()

df = env.get_fake_df()
first_with_gaming = pd.Series(group_fairness(df), name='first_with_gaming')
first_with_gaming

Consumer utility                  0.443801
Seller utility                    0.309059
average_num_got_shoes           193.000000
average_percent_got_shoes         0.021444
average_shoes_bought            293.000000
average_shoes_per_person          0.032556
abnormal_num_got_shoes           28.000000
abnormal_percent_got_shoes        0.028000
abnormal_shoes_bought            43.000000
abnormal_shoes_per_person         0.043000
special_num_got_shoes             7.000000
special_percent_got_shoes         0.028000
special_shoes_bought             14.000000
special_shoes_per_person          0.056000
wealthy_num_got_shoes            14.000000
wealthy_percent_got_shoes         0.014000
wealthy_shoes_bought             21.000000
wealthy_shoes_per_person          0.021000
influencer_num_got_shoes         13.000000
influencer_percent_got_shoes      0.026000
influencer_shoes_bought          13.000000
influencer_shoes_per_person       0.026000
reseller_num_got_shoes          308.000000
reseller_pe

### Invitation Only
All influencers are invited, and the rest of the spots are randomly selected from those with loyalty status

In [11]:
env.restock(num_shoes = 1000)
env.run_invitation_with_gaming()
df = env.get_fake_df()
invite_with_gaming = pd.Series(group_fairness(df), name='invite_with_gaming')
invite_with_gaming

Consumer utility                  0.370729
Seller utility                    0.791775
average_num_got_shoes            44.000000
average_percent_got_shoes         0.004889
average_shoes_bought             63.000000
average_shoes_per_person          0.007000
abnormal_num_got_shoes            9.000000
abnormal_percent_got_shoes        0.009000
abnormal_shoes_bought            15.000000
abnormal_shoes_per_person         0.015000
special_num_got_shoes             0.000000
special_percent_got_shoes         0.000000
special_shoes_bought              0.000000
special_shoes_per_person          0.000000
wealthy_num_got_shoes             9.000000
wealthy_percent_got_shoes         0.009000
wealthy_shoes_bought             14.000000
wealthy_shoes_per_person          0.014000
influencer_num_got_shoes        500.000000
influencer_percent_got_shoes      1.000000
influencer_shoes_bought         500.000000
influencer_shoes_per_person       1.000000
reseller_num_got_shoes          204.000000
reseller_pe

## Change cap to 1
Change the cap to one to prevent resellers get too many shoes and ensure the fairness

In [12]:
env.restock(1000)
env.run_lottery_with_gaming(cap=1)  # run lottery and get dataframe of all results
df = env.get_fake_df()
lottery_cap1_with_gaming = pd.Series(group_fairness(df), name='lottery_cap1_with_gaming')

## Authentication

With auth, it's harder for reseller get fake acccounts

In [13]:
env.restock(1000)
env.run_lottery_with_gaming_auth(cap=2)  # run lottery and get dataframe of all results
df = env.get_fake_df()
lottery_authen_with_gaming = pd.Series(group_fairness(df), name='lottery_authen_with_gaming')
lottery_authen_with_gaming

Consumer utility                  0.202509
Seller utility                    0.342806
average_num_got_shoes           160.000000
average_percent_got_shoes         0.017778
average_shoes_bought            233.000000
average_shoes_per_person          0.025889
abnormal_num_got_shoes           22.000000
abnormal_percent_got_shoes        0.022000
abnormal_shoes_bought            32.000000
abnormal_shoes_per_person         0.032000
special_num_got_shoes             6.000000
special_percent_got_shoes         0.024000
special_shoes_bought             12.000000
special_shoes_per_person          0.048000
wealthy_num_got_shoes            26.000000
wealthy_percent_got_shoes         0.026000
wealthy_shoes_bought             37.000000
wealthy_shoes_per_person          0.037000
influencer_num_got_shoes         11.000000
influencer_percent_got_shoes      0.022000
influencer_shoes_bought          11.000000
influencer_shoes_per_person       0.022000
reseller_num_got_shoes          338.000000
reseller_pe

In [17]:
env.restock(1000)
env.run_lottery_with_gaming_auth(cap=1)  # run lottery and get dataframe of all results
df = env.get_fake_df()
lottery_authen_cap1_with_gaming = pd.Series(group_fairness(df), name='lottery_authen_cap1_with_gaming')
lottery_authen_cap1_with_gaming

Consumer utility                  0.177405
Seller utility                    0.352376
average_num_got_shoes           277.000000
average_percent_got_shoes         0.030778
average_shoes_bought            277.000000
average_shoes_per_person          0.030778
abnormal_num_got_shoes           29.000000
abnormal_percent_got_shoes        0.029000
abnormal_shoes_bought            29.000000
abnormal_shoes_per_person         0.029000
special_num_got_shoes             0.000000
special_percent_got_shoes         0.000000
special_shoes_bought              0.000000
special_shoes_per_person          0.000000
wealthy_num_got_shoes            31.000000
wealthy_percent_got_shoes         0.031000
wealthy_shoes_bought             31.000000
wealthy_shoes_per_person          0.031000
influencer_num_got_shoes         14.000000
influencer_percent_got_shoes      0.028000
influencer_shoes_bought          14.000000
influencer_shoes_per_person       0.028000
reseller_num_got_shoes          638.000000
reseller_pe

## Combine three distribution methods

Since it's easier for resellers to use bot online, shoes will be distributed in a proportion that fewer percentage of them can be bought online to prevent that

In [19]:
env.restock(200)
env.run_lottery_with_gaming_auth()
env.restock_without_reset(500)
env.run_first_come_with_gaming()
env.restock_without_reset(300)
env.run_invitation_with_gaming()
df = env.get_fake_df()
combined_distribution_with_gaming = pd.Series(group_fairness(df), name='combined_distribution_with_gaming')
combined_distribution_with_gaming

Consumer utility                  0.433763
Seller utility                    0.612482
average_num_got_shoes           128.000000
average_percent_got_shoes         0.014222
average_shoes_bought            195.000000
average_shoes_per_person          0.021667
abnormal_num_got_shoes           16.000000
abnormal_percent_got_shoes        0.016000
abnormal_shoes_bought            24.000000
abnormal_shoes_per_person         0.024000
special_num_got_shoes             3.000000
special_percent_got_shoes         0.012000
special_shoes_bought              6.000000
special_shoes_per_person          0.024000
wealthy_num_got_shoes            15.000000
wealthy_percent_got_shoes         0.015000
wealthy_shoes_bought             20.000000
wealthy_shoes_per_person          0.020000
influencer_num_got_shoes        302.000000
influencer_percent_got_shoes      0.604000
influencer_shoes_bought         307.000000
influencer_shoes_per_person       0.614000
reseller_num_got_shoes          225.000000
reseller_pe

## Get Final Results

In [20]:
final_result = pd.DataFrame([lottery_no_gaming, first_no_gaming, invite_no_gaming,lottery_with_gaming,first_with_gaming,invite_with_gaming,lottery_cap1_with_gaming,lottery_authen_with_gaming,lottery_authen_cap1_with_gaming,combined_distribution_with_gaming])

In [21]:
# uncomment to save results
# final_result.to_csv('project2_results.txt',sep = '\t') 
final_result

,Consumer utility,Seller utility,average_num_got_shoes,average_percent_got_shoes,average_shoes_bought,average_shoes_per_person,abnormal_num_got_shoes,abnormal_percent_got_shoes,abnormal_shoes_bought,abnormal_shoes_per_person,...,wealthy_shoes_bought,wealthy_shoes_per_person,influencer_num_got_shoes,influencer_percent_got_shoes,influencer_shoes_bought,influencer_shoes_per_person,reseller_num_got_shoes,reseller_percent_got_shoes,reseller_shoes_bought,reseller_shoes_per_person
lottery_no_gaming,0.420476,0.162939,396.0,0.044000,613.0,0.068111,41.0,0.041,63.0,0.063,...,80.0,0.080,24.0,0.048,24.0,0.048,101.0,0.0404,202.0,0.0808
first_no_gaming,0.905261,0.073881,526.0,0.058444,786.0,0.087333,63.0,0.063,99.0,0.099,...,61.0,0.061,30.0,0.060,30.0,0.060,0.0,0.0000,0.0,0.0000
invite_no_gaming,0.451438,0.705031,154.0,0.017111,242.0,0.026889,16.0,0.016,25.0,0.025,...,45.0,0.045,500.0,1.000,500.0,1.000,90.0,0.0360,180.0,0.0720
lottery_with_gaming,0.104472,0.416667,84.0,0.009333,128.0,0.014222,6.0,0.006,8.0,0.008,...,23.0,0.023,6.0,0.012,6.0,0.012,414.0,0.1656,827.0,0.3308
first_with_gaming,0.443801,0.309059,193.0,0.021444,293.0,0.032556,28.0,0.028,43.0,0.043,...,21.0,0.021,13.0,0.026,13.0,0.026,308.0,0.1232,616.0,0.2464
invite_with_gaming,0.370729,0.791775,44.0,0.004889,63.0,0.007000,9.0,0.009,15.0,0.015,...,14.0,0.014,500.0,1.000,500.0,1.000,204.0,0.0816,408.0,0.1632
lottery_cap1_with_gaming,0.072467,0.442211,112.0,0.012444,112.0,0.012444,10.0,0.010,10.0,0.010,...,12.0,0.012,7.0,0.014,7.0,0.014,854.0,0.3416,854.0,0.3416
lottery_authen_with_gaming,0.202509,0.342806,160.0,0.017778,233.0,0.025889,22.0,0.022,32.0,0.032,...,37.0,0.037,11.0,0.022,11.0,0.022,338.0,0.1352,675.0,0.2700
lottery_authen_cap1_with_gaming,0.177405,0.352376,277.0,0.030778,277.0,0.030778,29.0,0.029,29.0,0.029,...,31.0,0.031,14.0,0.028,14.0,0.028,638.0,0.2552,638.0,0.2552
combined_distribution_with_gaming,0.433763,0.612482,128.0,0.014222,195.0,0.021667,16.0,0.016,24.0,0.024,...,20.0,0.020,302.0,0.604,307.0,0.614,225.0,0.0900,448.0,0.1792
